# Azure AI Agent service - Function calling

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [1]:
#%pip install azure-ai-projects
#%pip install yfinance

In [2]:
import glob
import json
import ipyplot
import matplotlib.pyplot as plt
import os
import requests
import sys
import time
import yfinance as yf

#from azure.ai.projects import AIProjectClient
from azure.ai.agents import AgentsClient 
from azure.identity import DefaultAzureCredential
#from azure.ai.projects.models import FunctionTool, ToolSet, CodeInterpreterTool
from azure.ai.agents.models import (
    FunctionTool,
    ListSortOrder,
    RequiredFunctionToolCall,
    SubmitToolOutputsAction,
    ToolOutput,
)
from datetime import datetime, timezone, timedelta
from dotenv import load_dotenv
from openai import AzureOpenAI
from typing import Any, Callable, Set, Dict, List, Optional
from PIL import Image
from pathlib import Path

from user_functions import user_functions, azuremaps_weather

In [3]:
load_dotenv("azure.env")

True

## 1. Agent definition - Weather forecasts

## Connect to AI Foundry Project

In [4]:
endpoint = os.getenv("PROJECT_ENDPOINT")
print(f"Using endpoint: {endpoint}")
credential = DefaultAzureCredential()

project_client = AgentsClient(endpoint=endpoint, credential=credential)

Using endpoint: https://aq-ai-foundry-sweden-central.services.ai.azure.com/api/projects/firstProject


### Testing the Azure Maps Weather function

We will use the Azure Maps service to provide weather information.

In [5]:
weather = azuremaps_weather("Toronto")
json.loads(weather)

{'weather_data': {'results': [{'dateTime': '2025-06-09T10:22:00-04:00',
    'phrase': 'Cloudy',
    'iconCode': 7,
    'hasPrecipitation': False,
    'isDayTime': True,
    'temperature': {'value': 16.1, 'unit': 'C', 'unitType': 17},
    'realFeelTemperature': {'value': 18.7, 'unit': 'C', 'unitType': 17},
    'realFeelTemperatureShade': {'value': 16.7, 'unit': 'C', 'unitType': 17},
    'relativeHumidity': 93,
    'dewPoint': {'value': 15.0, 'unit': 'C', 'unitType': 17},
    'wind': {'direction': {'degrees': 0.0, 'localizedDescription': 'N'},
     'speed': {'value': 3.7, 'unit': 'km/h', 'unitType': 7}},
    'windGust': {'speed': {'value': 8.9, 'unit': 'km/h', 'unitType': 7}},
    'uvIndex': 2,
    'uvIndexPhrase': 'Low',
    'visibility': {'value': 14.5, 'unit': 'km', 'unitType': 6},
    'obstructionsToVisibility': '',
    'cloudCover': 100,
    'ceiling': {'value': 488.0, 'unit': 'm', 'unitType': 5},
    'pressure': {'value': 1006.4, 'unit': 'mb', 'unitType': 14},
    'pressureTendency

### Agent

In [6]:
model = "gpt-4o-mini"
name = "gpt4o-weather-agent"
instructions = "You are a weather bot. Use the provided functions to help answer questions."

functions = FunctionTool(user_functions)

# Create an agent and run user's request with function calls
agent = project_client.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="my-agent",
    instructions="You are a helpful agent",
    tools=functions.definitions,
)
print(f"Created agent, ID: {agent.id}")



Created agent, ID: asst_vXM2Gy2n7bx4XjdJYl8dZGUI


### Tests

In [7]:
prompt = "Hello, generate a full report for the weather in Toronto today"

# Create thread for communication
thread = project_client.threads.create()
print(f"Created thread ID = {thread.id}")

# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

print(f"Created message ID = {message.id}")

Created thread ID = thread_pDppDIv73MmIgPS7pDrQXCUx
Created message ID = msg_Sidn3ZmsICsu4jE5NvxNwqEq


In [8]:
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"Created run, run ID: {run.id}")

Created run, run ID: run_QsawhITSHIKmhCVusEIJRpvr


In [9]:
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = project_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_outputs = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredFunctionToolCall):
                    try:
                        print(f"Executing tool call: {tool_call}")
                        output = functions.execute(tool_call)
                        tool_outputs.append(
                            ToolOutput(
                                tool_call_id=tool_call.id,
                                output=output,
                            )
                        )
                    except Exception as e:
                        print(f"Error executing tool_call {tool_call.id}: {e}")

            print(f"Tool outputs: {tool_outputs}")
            if tool_outputs:
                project_client.runs.submit_tool_outputs(thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs)

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")

Executing tool call: {'id': 'call_11WTKw5ZvWRrU6psjQiRYfKi', 'type': 'function', 'function': {'name': 'azuremaps_weather', 'arguments': '{"query":"Toronto"}'}}
Tool outputs: [{'tool_call_id': 'call_11WTKw5ZvWRrU6psjQiRYfKi', 'output': '{"weather_data": {"results": [{"dateTime": "2025-06-09T10:22:00-04:00", "phrase": "Cloudy", "iconCode": 7, "hasPrecipitation": false, "isDayTime": true, "temperature": {"value": 16.1, "unit": "C", "unitType": 17}, "realFeelTemperature": {"value": 18.7, "unit": "C", "unitType": 17}, "realFeelTemperatureShade": {"value": 16.7, "unit": "C", "unitType": 17}, "relativeHumidity": 93, "dewPoint": {"value": 15.0, "unit": "C", "unitType": 17}, "wind": {"direction": {"degrees": 0.0, "localizedDescription": "N"}, "speed": {"value": 3.7, "unit": "km/h", "unitType": 7}}, "windGust": {"speed": {"value": 8.9, "unit": "km/h", "unitType": 7}}, "uvIndex": 2, "uvIndexPhrase": "Low", "visibility": {"value": 14.5, "unit": "km", "unitType": 6}, "obstructionsToVisibility": "",

In [10]:
start   = run.started_at       
end     = run.completed_at
elapsed = end - start

iso_fmt = "%Y-%m-%d %H:%M:%S%z"
print(f"Start   : {start.strftime(iso_fmt)}")
print(f"End     : {end.strftime(iso_fmt)}")
print(f"Elapsed : {elapsed}  "
      f"({elapsed.total_seconds():.2f} seconds)")

run.usage

Start   : 2025-06-09 14:31:34+0000
End     : 2025-06-09 14:31:39+0000
Elapsed : 0:00:05  (5.00 seconds)


{'prompt_tokens': 1407, 'completion_tokens': 395, 'total_tokens': 1802, 'prompt_token_details': {'cached_tokens': 0}}

In [11]:
    # Fetch and log all messages
    messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role}: {last_text.text.value}")

MessageRole.USER: Hello, generate a full report for the weather in Toronto today
MessageRole.AGENT: Here is a full weather report for Toronto today:

Current Conditions (as of 10:22 AM, June 9, 2025):

- Weather: Cloudy
- Temperature: 16.1°C
- RealFeel® Temperature: 18.7°C
- RealFeel® in Shade: 16.7°C
- Apparent Temperature: 19.4°C
- Wind Chill: 16.1°C
- Relative Humidity: 93%
- Dew Point: 15.0°C
- Wind: North at 3.7 km/h
- Wind Gusts: Up to 8.9 km/h
- UV Index: 2 (Low)
- Visibility: 14.5 km
- Ceiling (cloud base): 488 meters
- Cloud Cover: 100%
- Pressure: 1006.4 mb (falling)

Precipitation:

- No precipitation in the past hour, 3 hours, 6 hours, 9 hours, or 12 hours
- 1.1 mm in the past 18 hours and 24 hours

Temperature Summary:

- Past 6 hours: Min 14.9°C, Max 16.1°C
- Past 12 hours: Min 13.9°C, Max 16.1°C
- Past 24 hours: Min 13.9°C, Max 20.0°C

Remarks:

- The weather is currently cloudy with high humidity, making it feel slightly warmer than the actual temperature.
- There has b

### Other examples

In [12]:
prompt = "Hello, generate a full report for the weather in Mexico City today. Print the results in English, Spanish and French."

# Create thread for communication
thread = project_client.threads.create()
print(f"Created thread ID = {thread.id}")

# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

print(f"Created message ID = {message.id}")

Created thread ID = thread_lg2c6Ef5Yo90zik72JJ8ho89
Created message ID = msg_hNc0r6tlGcdqrenurvTTVPgC


In [13]:
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"Created run, run ID: {run.id}")

Created run, run ID: run_kldvLG3jvzOGt6JT9aZzYsnz


In [14]:
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = project_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_outputs = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredFunctionToolCall):
                    try:
                        print(f"Executing tool call: {tool_call}")
                        output = functions.execute(tool_call)
                        tool_outputs.append(
                            ToolOutput(
                                tool_call_id=tool_call.id,
                                output=output,
                            )
                        )
                    except Exception as e:
                        print(f"Error executing tool_call {tool_call.id}: {e}")

            print(f"Tool outputs: {tool_outputs}")
            if tool_outputs:
                project_client.runs.submit_tool_outputs(thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs)

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")

Executing tool call: {'id': 'call_nxUYfmvtri5CcZtFwwF3FAza', 'type': 'function', 'function': {'name': 'azuremaps_weather', 'arguments': '{"query":"Mexico City"}'}}
Tool outputs: [{'tool_call_id': 'call_nxUYfmvtri5CcZtFwwF3FAza', 'output': '{"weather_data": {"results": [{"dateTime": "2025-06-09T08:20:00-06:00", "phrase": "Sunny", "iconCode": 1, "hasPrecipitation": false, "isDayTime": true, "temperature": {"value": 20.0, "unit": "C", "unitType": 17}, "realFeelTemperature": {"value": 22.2, "unit": "C", "unitType": 17}, "realFeelTemperatureShade": {"value": 18.8, "unit": "C", "unitType": 17}, "relativeHumidity": 59, "dewPoint": {"value": 12.2, "unit": "C", "unitType": 17}, "wind": {"direction": {"degrees": 225.0, "localizedDescription": "SW"}, "speed": {"value": 7.4, "unit": "km/h", "unitType": 7}}, "windGust": {"speed": {"value": 7.4, "unit": "km/h", "unitType": 7}}, "uvIndex": 3, "uvIndexPhrase": "Moderate", "visibility": {"value": 11.3, "unit": "km", "unitType": 6}, "obstructionsToVisib

In [15]:
start   = run.started_at       
end     = run.completed_at
elapsed = end - start

iso_fmt = "%Y-%m-%d %H:%M:%S%z"
print(f"Start   : {start.strftime(iso_fmt)}")
print(f"End     : {end.strftime(iso_fmt)}")
print(f"Elapsed : {elapsed}  "
      f"({elapsed.total_seconds():.2f} seconds)")

run.usage

Start   : 2025-06-09 14:31:45+0000
End     : 2025-06-09 14:31:50+0000
Elapsed : 0:00:05  (5.00 seconds)


{'prompt_tokens': 1433, 'completion_tokens': 573, 'total_tokens': 2006, 'prompt_token_details': {'cached_tokens': 0}}

In [16]:
    # Fetch and log all messages
    messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role}: {last_text.text.value}")

MessageRole.USER: Hello, generate a full report for the weather in Mexico City today. Print the results in English, Spanish and French.
MessageRole.AGENT: Here is the full weather report for Mexico City today, presented in English, Spanish, and French:

English:
- Current conditions: Sunny
- Temperature: 20.0°C (feels like 22.2°C)
- In the shade: feels like 18.8°C
- Relative humidity: 59%
- Dew point: 12.2°C
- Wind: Southwest at 7.4 km/h (gusts up to 7.4 km/h)
- UV Index: 3 (Moderate)
- Visibility: 11.3 km
- Cloud cover: 0%
- Atmospheric pressure: 1027.1 mb (Rising)
- Precipitation (past 24 hours): 0.0 mm
- Temperature range (last 24 hours): Min 14.9°C, Max 27.2°C

Spanish:
- Condiciones actuales: Soleado
- Temperatura: 20.0°C (sensación térmica de 22.2°C)
- A la sombra: sensación de 18.8°C
- Humedad relativa: 59%
- Punto de rocío: 12.2°C
- Viento: Suroeste a 7.4 km/h (rachas hasta 7.4 km/h)
- Índice UV: 3 (Moderado)
- Visibilidad: 11.3 km
- Cobertura nubosa: 0%
- Presión atmosférica: 

In [17]:
prompt = "Hello, print the UV informations for Oakville, Ontario today. Print the date as well of the weather data."

# Create thread for communication
thread = project_client.threads.create()
print(f"Created thread ID = {thread.id}")

# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

print(f"Created message ID = {message.id}")

Created thread ID = thread_4cgvJYlD1NXD2tS62Zc79Sna
Created message ID = msg_HcorjFfJmI4yf0537b6P7e3s


In [18]:
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"Created run, run ID: {run.id}")

Created run, run ID: run_K8uLWLneFuTg3Fcg8a6z071v


In [19]:
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = project_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_outputs = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredFunctionToolCall):
                    try:
                        print(f"Executing tool call: {tool_call}")
                        output = functions.execute(tool_call)
                        tool_outputs.append(
                            ToolOutput(
                                tool_call_id=tool_call.id,
                                output=output,
                            )
                        )
                    except Exception as e:
                        print(f"Error executing tool_call {tool_call.id}: {e}")

            print(f"Tool outputs: {tool_outputs}")
            if tool_outputs:
                project_client.runs.submit_tool_outputs(thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs)

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")

Executing tool call: {'id': 'call_PwDzZHqjcRlRS6FVJuR27AAl', 'type': 'function', 'function': {'name': 'azuremaps_weather', 'arguments': '{"query":"Oakville, Ontario"}'}}
Tool outputs: [{'tool_call_id': 'call_PwDzZHqjcRlRS6FVJuR27AAl', 'output': '{"weather_data": {"results": [{"dateTime": "2025-06-09T10:17:00-04:00", "phrase": "Cloudy", "iconCode": 7, "hasPrecipitation": false, "isDayTime": true, "temperature": {"value": 17.5, "unit": "C", "unitType": 17}, "realFeelTemperature": {"value": 20.3, "unit": "C", "unitType": 17}, "realFeelTemperatureShade": {"value": 18.2, "unit": "C", "unitType": 17}, "relativeHumidity": 90, "dewPoint": {"value": 15.9, "unit": "C", "unitType": 17}, "wind": {"direction": {"degrees": 158.0, "localizedDescription": "SSE"}, "speed": {"value": 3.5, "unit": "km/h", "unitType": 7}}, "windGust": {"speed": {"value": 9.9, "unit": "km/h", "unitType": 7}}, "uvIndex": 2, "uvIndexPhrase": "Low", "visibility": {"value": 14.5, "unit": "km", "unitType": 6}, "obstructionsToVi

In [20]:
start   = run.started_at       
end     = run.completed_at
elapsed = end - start

iso_fmt = "%Y-%m-%d %H:%M:%S%z"
print(f"Start   : {start.strftime(iso_fmt)}")
print(f"End     : {end.strftime(iso_fmt)}")
print(f"Elapsed : {elapsed}  "
      f"({elapsed.total_seconds():.2f} seconds)")

run.usage

Start   : 2025-06-09 14:31:58+0000
End     : 2025-06-09 14:31:59+0000
Elapsed : 0:00:01  (1.00 seconds)


{'prompt_tokens': 1433, 'completion_tokens': 74, 'total_tokens': 1507, 'prompt_token_details': {'cached_tokens': 0}}

In [21]:
    # Fetch and log all messages
    messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role}: {last_text.text.value}")

MessageRole.USER: Hello, print the UV informations for Oakville, Ontario today. Print the date as well of the weather data.
MessageRole.AGENT: Here is the UV information for Oakville, Ontario today:

- Date of weather data: 2025-06-09
- UV Index: 2
- UV Index Description: Low

You can safely spend time outdoors with minimal UV protection required today.


In [22]:
prompt = "Generate a weather report for Rome. This report is for a blog publication. Add emojis and adopt a fun style."

# Create thread for communication
thread = project_client.threads.create()
print(f"Created thread ID = {thread.id}")

# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

print(f"Created message ID = {message.id}")

Created thread ID = thread_k96A1b6QB6uIbljXmPDdVSH2
Created message ID = msg_6svnYVAdKSIrwfEHftizeGoD


In [23]:
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"Created run, run ID: {run.id}")

Created run, run ID: run_ZQiqxY5q4Y98dG4tG2jTA741


In [24]:
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = project_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_outputs = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredFunctionToolCall):
                    try:
                        print(f"Executing tool call: {tool_call}")
                        output = functions.execute(tool_call)
                        tool_outputs.append(
                            ToolOutput(
                                tool_call_id=tool_call.id,
                                output=output,
                            )
                        )
                    except Exception as e:
                        print(f"Error executing tool_call {tool_call.id}: {e}")

            print(f"Tool outputs: {tool_outputs}")
            if tool_outputs:
                project_client.runs.submit_tool_outputs(thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs)

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")

Current run status: RunStatus.IN_PROGRESS
Executing tool call: {'id': 'call_1CzPrkBHXVA5oB3LtjktW91a', 'type': 'function', 'function': {'name': 'azuremaps_weather', 'arguments': '{"query":"Rome"}'}}
Tool outputs: [{'tool_call_id': 'call_1CzPrkBHXVA5oB3LtjktW91a', 'output': '{"weather_data": {"results": [{"dateTime": "2025-06-09T16:16:00+02:00", "phrase": "Sunny", "iconCode": 1, "hasPrecipitation": false, "isDayTime": true, "temperature": {"value": 27.4, "unit": "C", "unitType": 17}, "realFeelTemperature": {"value": 27.4, "unit": "C", "unitType": 17}, "realFeelTemperatureShade": {"value": 25.5, "unit": "C", "unitType": 17}, "relativeHumidity": 48, "dewPoint": {"value": 15.5, "unit": "C", "unitType": 17}, "wind": {"direction": {"degrees": 203.0, "localizedDescription": "SSW"}, "speed": {"value": 20.3, "unit": "km/h", "unitType": 7}}, "windGust": {"speed": {"value": 37.4, "unit": "km/h", "unitType": 7}}, "uvIndex": 3, "uvIndexPhrase": "Moderate", "visibility": {"value": 20.9, "unit": "km"

In [25]:
start   = run.started_at       
end     = run.completed_at
elapsed = end - start

iso_fmt = "%Y-%m-%d %H:%M:%S%z"
print(f"Start   : {start.strftime(iso_fmt)}")
print(f"End     : {end.strftime(iso_fmt)}")
print(f"Elapsed : {elapsed}  "
      f"({elapsed.total_seconds():.2f} seconds)")

run.usage

Start   : 2025-06-09 14:32:08+0000
End     : 2025-06-09 14:32:13+0000
Elapsed : 0:00:05  (5.00 seconds)


{'prompt_tokens': 1431, 'completion_tokens': 325, 'total_tokens': 1756, 'prompt_token_details': {'cached_tokens': 0}}

In [26]:
    # Fetch and log all messages
    messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role}: {last_text.text.value}")

MessageRole.USER: Generate a weather report for Rome. This report is for a blog publication. Add emojis and adopt a fun style.
MessageRole.AGENT: Buongiorno from the Eternal City! 🇮🇹🌞

Rome is putting its best face forward today, and wow, what a face it is! With gloriously sunny skies and a toasty temperature around 27°C (that’s beach weather, but in the city!) the city is ready for gelato, Vespa rides, and strolls past the Colosseum.

What’s happening in the sky? Absolutely nothing! Zero clouds—just pure sunshine ☀️. The sun is shining bright, so don’t forget your sunglasses and maybe channel your inner Roman gladiator with some SPF 50 🕶️🧴.

Humidity is sitting at a comfy 48%, which means the air feels just as lovely as a scoop of tiramisu. A gentle wind is breezing in from the south-southwest at a refreshing 20 km/h, with the occasional gust that might just give your hair that windswept Italian look 💨💁.

UV index is moderate (3/10), so soak up those rays—but maybe find a shady piazza

### Post processing